In [ ]:
import mlrose_hiive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
# define the nqueens problem and print an example
fitness = mlrose_hiive.Queens()
problem_sizes = range(4, 44, 4)
seed = 166

problems, states = list(), list()
for problem_size in problem_sizes:
    state = np.random.choice(list(range(problem_size)), size=problem_size)
    states.append(state)
#     print(state)
#     print(fitness.evaluate(state))
    problem = mlrose_hiive.DiscreteOpt(length = problem_size, fitness_fn = fitness, maximize = True, max_val = problem_size)
    problems.append(problem)

In [ ]:
RHC,SA,GA,MIMIC = list(),list(),list(),list()
for i in range(len(problems)):
    # Randomized Hill Climbing
    rhc_results = mlrose_hiive.random_hill_climb(problem=problems[i], init_state=states[i], max_attempts=100, max_iters=1000, random_state=seed, restarts=20)
    RHC.append(rhc_results[1])
    print("RHC done: ", problem_sizes[i])
    # Simulated Annealing
    sa_results = mlrose_hiive.simulated_annealing(problems[i], schedule=mlrose_hiive.ExpDecay(init_temp=2.0), init_state=states[i], max_attempts=1000, max_iters=3000, random_state=seed)
    SA.append(sa_results[1])
    print("SA done: ", problem_sizes[i])
    # Generic Algorithm
    ga_results = mlrose_hiive.genetic_alg(problems[i], max_attempts=100, max_iters=1000,mutation_prob=0.5, random_state=seed)
    GA.append(ga_results[1])
    print("GA done: ", problem_sizes[i])
    mimic_results = mlrose_hiive.mimic(problems[i], max_attempts=100, max_iters=1000,keep_pct=0.7, random_state=seed)
    MIMIC.append(mimic_results[1])
    print("MIMIC done: ", problem_sizes[i])

In [ ]:
# plot the "problem sizes-best fitness score"
data = {'Problem_Size': range(4, 44, 4),
        'RHC': RHC,
        'SA': SA,
        'GA': GA,
        'MIMIC': MIMIC,
        }
df = pd.DataFrame(data, columns=['Problem_Size', 'RHC', 'SA', 'GA', 'MIMIC'])

print("Nqueens results")
print(df)

ax = plt.gca()
df.plot(kind='line', x='Problem_Size', y='RHC', marker='o', ax=ax)
df.plot(kind='line', x='Problem_Size', y='SA', marker='o', ax=ax)
df.plot(kind='line', x='Problem_Size', y='GA', marker='o', ax=ax)
df.plot(kind='line', x='Problem_Size', y='MIMIC', marker='o', ax=ax)
 
plt.xlabel('Problem Size')
plt.ylabel('Fitness Values')
plt.title('Randomized Optimization for Nqueens problem')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\Nqueens Performance.png', dpi=300)
plt.show()

In [ ]:
# choose the problem size that differentiate different algorithms
state = states[8]
problem = mlrose_hiive.DiscreteOpt(length = 36, fitness_fn = fitness, maximize = True, max_val = 36)

In [ ]:
# randomized hill climbing - tuning parameter
nums = [10, 25, 50, 75, 100]
restarts_result = list()
for num in nums:
    rhc_results = mlrose_hiive.random_hill_climb(problem=problem, init_state=state, max_attempts=100, max_iters=3000, random_state=seed, restarts=num, curve=True)
    restarts_result.append(rhc_results[2])
    print(rhc_results[1], rhc_results[2].shape)

for i in range(len(restarts_result)):
    array = restarts_result[i]
    # subtract
    array[:, 1] = array[:,1] - array[0, 1]

    current_length = array.shape[0]
    if current_length < 1800:
        append_array = np.array([[array[-1, 0], array[-1, 1]] for _ in range(1800-current_length)])
        temp = np.concatenate((array, append_array), axis=0)
        restarts_result[i] = temp

dic = {'iterations': range(1800),
        '10': restarts_result[0][:, 0],
        '25': restarts_result[1][:, 0],
        '50': restarts_result[2][:, 0],
        '75': restarts_result[3][:, 0],
        '100': restarts_result[4][:, 0],
        }
df = pd.DataFrame(dic, columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='10', ax=ax)
df.plot(kind='line', x='iterations', y='25',  ax=ax)
df.plot(kind='line', x='iterations', y='50',  ax=ax)
df.plot(kind='line', x='iterations', y='75', ax=ax)
df.plot(kind='line', x='iterations', y='100', ax=ax)

 
plt.xlabel('Iterations')
plt.ylabel('Fitness Values')
plt.title('RHC - Iterations vs Fitness Values (restarts)')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueens-rhc1.png', dpi=500)
plt.show()

dic = {'iterations': range(1800),
        '10': restarts_result[0][:, 1],
        '25': restarts_result[1][:, 1],
        '50': restarts_result[2][:, 1],
        '75': restarts_result[3][:, 1],
        '100': restarts_result[4][:, 1],
        }
df = pd.DataFrame(dic, columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='10', ax=ax)
df.plot(kind='line', x='iterations', y='25',  ax=ax)
df.plot(kind='line', x='iterations', y='50',  ax=ax)
df.plot(kind='line', x='iterations', y='75', ax=ax)
df.plot(kind='line', x='iterations', y='100', ax=ax)

 
plt.xlabel('Iterations')
plt.ylabel('Cumulative number of function evaluation')
plt.title('RHC - Cumulative number of function evaluation')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueens-rhc2.png', dpi=500)
plt.show()

In [ ]:
# rhc - best parameter, test time
t0_clock = time.process_time()
rhc_results = mlrose_hiive.random_hill_climb(problem, init_state=state, max_attempts=100, max_iters=3000, random_state=seed, restarts=50, curve=True)
t1_clock = time.process_time()
print("fitness score: ", rhc_results[1])
print("# iterations: ", rhc_results[2].shape[0])
print("# cumulative function iterations: ", rhc_results[2][-1,1] - rhc_results[2][0,1])
print("time_consumed: ", t1_clock - t0_clock)

In [ ]:
# simulated annealing - tuning parameter - strategy and temperature
options = [mlrose_hiive.GeomDecay(init_temp=1), mlrose_hiive.ArithDecay(init_temp=1), mlrose_hiive.ExpDecay(init_temp=1)]
schedule_result = list()
for option in options:
    sa_results = mlrose_hiive.simulated_annealing(problem, schedule=option, init_state=state, max_attempts=100, max_iters=3000, random_state=seed, curve=True)
    schedule_result.append(sa_results[2])
    print(sa_results[1], sa_results[2].shape)
    
for i in range(len(schedule_result)):
    array = schedule_result[i]
    # subtract
    array[:, 1] = array[:, 1] - array[0, 1]

    current_length = array.shape[0]
    if current_length < 3000:
        append_array = np.array([[array[-1, 0], array[-1, 1]] for _ in range(3000-current_length)])
        temp = np.concatenate((array, append_array), axis=0)
        schedule_result[i] = temp

dic = {'iterations': range(3000),
        'GeomDecay-1': schedule_result[0][:, 0],
        'ArithDecay-1': schedule_result[1][:, 0],
        'ExpDecay-1': schedule_result[2][:, 0],
      }
df = pd.DataFrame(dic,columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='GeomDecay-1', ax=ax)
df.plot(kind='line', x='iterations', y='ArithDecay-1',  ax=ax)
df.plot(kind='line', x='iterations', y='ExpDecay-1',  ax=ax)
 
plt.xlabel('Iterations')
plt.ylabel('Fitness Values')
plt.title('Simulated Annealing - different schedule')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueens-sa1.png', dpi=500)
plt.show()

In [ ]:
# simulated annealing - tuning parameter
temperatures = [0.5, 1, 2, 5, 10]
temperature_result = list()
for temperature in temperatures:
    sa_results = mlrose_hiive.simulated_annealing(problem, schedule=mlrose_hiive.ArithDecay(init_temp=temperature), init_state=state, max_attempts=100, max_iters=3000, random_state=seed, curve=True)
    temperature_result.append(sa_results[2])
    print(sa_results[1], sa_results[2].shape)
    
for i in range(len(temperature_result)):
    array = temperature_result[i]
    # subtract
    array[:, 1] = array[:,1] - array[0, 1]

    current_length = array.shape[0]
    if current_length < 3000:
        append_array = np.array([[array[-1, 0], array[-1, 1]] for _ in range(3000-current_length)])
        temp = np.concatenate((array, append_array), axis=0)
        temperature_result[i] = temp

dic = {'iterations': range(3000),
        '0.5': temperature_result[0][:, 0],
        '1': temperature_result[1][:, 0],
        '2': temperature_result[2][:, 0],
       '5': temperature_result[3][:, 0],
       '10': temperature_result[4][:, 0],
      }
df = pd.DataFrame(dic,columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='0.5', ax=ax)
df.plot(kind='line', x='iterations', y='1',  ax=ax)
df.plot(kind='line', x='iterations', y='2',  ax=ax)
df.plot(kind='line', x='iterations', y='5',  ax=ax)
df.plot(kind='line', x='iterations', y='10',  ax=ax)
 
plt.xlabel('Iterations')
plt.ylabel('Fitness Values')
plt.title('Simulated Annealing - tuning init temperature')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueen-sa2.png', dpi=500)
plt.show()

In [ ]:
# sa - best parameter, test time
t0_clock = time.process_time()
sa_results = mlrose_hiive.simulated_annealing(problem, schedule=mlrose_hiive.ArithDecay(init_temp=1), init_state=state, max_attempts=100, max_iters=3000, random_state=seed, curve=True)
t1_clock = time.process_time()
print("fitness score: ", sa_results[1])
print("# iterations: ", sa_results[2].shape[0])
print("# cumulative function iterations: ", sa_results[2][-1,1] - sa_results[2][0,1])
print("time_consumed: ", t1_clock - t0_clock)

In [ ]:
# generitic algorithm - tuning parameter - pop size and mutation prob
population = [25, 50, 100, 250, 500]
pop_result = list()
for pop in population:
    ga_results = mlrose_hiive.genetic_alg(problem, max_attempts=100, max_iters=1000, pop_size=pop, random_state=seed, curve=True)
    pop_result.append(ga_results[2])
    print(ga_results[1], ga_results[2].shape)
    
for i in range(len(pop_result)):
    array = pop_result[i]
    # subtract
    array[:, 1] = array[:, 1] - array[0, 1]

    current_length = array.shape[0]
    if current_length < 1000:
        append_array = np.array([[array[-1, 0], array[-1, 1]] for _ in range(1000-current_length)])
        temp = np.concatenate((array, append_array), axis=0)
        pop_result[i] = temp

dic = {'iterations': range(1000),
        '25': pop_result[0][:, 0],
        '50': pop_result[1][:, 0],
        '100': pop_result[2][:, 0],
       '200': pop_result[3][:, 0],
        '500': pop_result[4][:, 0],
      }
df = pd.DataFrame(dic,columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='25', ax=ax)
df.plot(kind='line', x='iterations', y='50',  ax=ax)
df.plot(kind='line', x='iterations', y='100',  ax=ax)
df.plot(kind='line', x='iterations', y='200',  ax=ax)
df.plot(kind='line', x='iterations', y='500',  ax=ax)
 
plt.xlabel('Iterations')
plt.ylabel('Fitness Values')
plt.title('Generic Algorithm - tuning population size')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueens-ga1.png', dpi=500)
plt.show()

In [ ]:
# generic algorithm - tuning parameter
ratios = [0.1, 0.2, 0.4, 0.6, 0.8]
mutation_result = list()
for ratio in ratios:
    ga_results = mlrose_hiive.genetic_alg(problem, max_attempts=100, max_iters=1000, pop_size=500, mutation_prob=ratio, random_state=seed, curve=True)
    mutation_result.append(ga_results[2])
    print(ga_results[1], ga_results[2].shape)
    
for i in range(len(mutation_result)):
    array = mutation_result[i]
    # subtract
    array[:, 1] = array[:,1] - array[0, 1]

    current_length = array.shape[0]
    if current_length < 1000:
        append_array = np.array([[array[-1, 0], array[-1, 1]] for _ in range(1000-current_length)])
        temp = np.concatenate((array, append_array), axis=0)
        mutation_result[i] = temp

dic = {'iterations': range(1000),
        '0.1': mutation_result[0][:, 0],
        '0.2': mutation_result[1][:, 0],
        '0.4': mutation_result[2][:, 0],
       '0.6': mutation_result[3][:, 0],
       '0.8': mutation_result[4][:, 0],
      }
df = pd.DataFrame(dic,columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='0.1', ax=ax)
df.plot(kind='line', x='iterations', y='0.2',  ax=ax)
df.plot(kind='line', x='iterations', y='0.4',  ax=ax)
df.plot(kind='line', x='iterations', y='0.6',  ax=ax)
df.plot(kind='line', x='iterations', y='0.8',  ax=ax)
 
plt.xlabel('Iterations')
plt.ylabel('Fitness Values')
plt.title('Generic Algorithm - tuning mutation probability')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueens-ga2.png', dpi=500)
plt.show()

In [ ]:
# ga - best parameter, test time
t0_clock = time.process_time()
ga_results = mlrose_hiive.genetic_alg(problem, max_attempts=100, max_iters=1000, pop_size=500, mutation_prob=0.4, random_state=seed, curve=True)
t1_clock = time.process_time()
print("fitness score: ", ga_results[1])
print("# iterations: ", ga_results[2].shape[0])
print("# cumulative function iterations: ", ga_results[2][-1,1] - ga_results[2][0,1])
print("time_consumed: ", t1_clock - t0_clock)

In [ ]:
# MIMIC - tuning parameter
population = [25, 50, 100, 250, 500]
pop_result = list()
for pop in population:
    mimic_results = mlrose_hiive.mimic(problem, max_attempts=100, max_iters=1000, pop_size=pop, random_state=seed, curve=True)
    pop_result.append(mimic_results[2])
    print(mimic_results[1], mimic_results[2].shape)
    
for i in range(len(pop_result)):
    array = pop_result[i]
    # subtract
    array[:, 1] = array[:, 1] - array[0, 1]

    current_length = array.shape[0]
    if current_length < 200:
        append_array = np.array([[array[-1, 0], array[-1, 1]] for _ in range(200-current_length)])
        temp = np.concatenate((array, append_array), axis=0)
        pop_result[i] = temp

dic = {'iterations': range(200),
        '25': pop_result[0][:, 0],
        '50': pop_result[1][:, 0],
        '100': pop_result[2][:, 0],
       '200': pop_result[3][:, 0],
        '500': pop_result[4][:, 0],
      }
df = pd.DataFrame(dic,columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='25', ax=ax)
df.plot(kind='line', x='iterations', y='50',  ax=ax)
df.plot(kind='line', x='iterations', y='100',  ax=ax)
df.plot(kind='line', x='iterations', y='200',  ax=ax)
df.plot(kind='line', x='iterations', y='500',  ax=ax)
 
plt.xlabel('Iterations')
plt.ylabel('Fitness Values')
plt.title('MIMIC - tuning population size')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueens-mimic1.png', dpi=500)
plt.show()

In [ ]:
# MIMIC - tuning parameter
ratios = [0.1, 0.2, 0.4, 0.6, 0.8]
mutation_result = list()
for ratio in ratios:
    mimic_results = mlrose_hiive.mimic(problem, max_attempts=100, max_iters=1000, pop_size=500, keep_pct=ratio, random_state=seed, curve=True)
    mutation_result.append(mimic_results[2])
    print(mimic_results[1], mimic_results[2].shape)
    
for i in range(len(mutation_result)):
    array = mutation_result[i]
    # subtract
    array[:, 1] = array[:,1] - array[0, 1]

    current_length = array.shape[0]
    if current_length < 300:
        append_array = np.array([[array[-1, 0], array[-1, 1]] for _ in range(300-current_length)])
        temp = np.concatenate((array, append_array), axis=0)
        mutation_result[i] = temp

dic = {'iterations': range(300),
        '0.1': mutation_result[0][:, 0],
        '0.2': mutation_result[1][:, 0],
        '0.4': mutation_result[2][:, 0],
       '0.6': mutation_result[3][:, 0],
       '0.8': mutation_result[4][:, 0],
      }
df = pd.DataFrame(dic,columns=dic.keys())

print("4 PEAKS result")
print(df)

ax = plt.gca()
df.plot(kind='line', x='iterations', y='0.1', ax=ax)
df.plot(kind='line', x='iterations', y='0.2',  ax=ax)
df.plot(kind='line', x='iterations', y='0.4',  ax=ax)
df.plot(kind='line', x='iterations', y='0.6',  ax=ax)
df.plot(kind='line', x='iterations', y='0.8',  ax=ax)
 
plt.xlabel('Iterations')
plt.ylabel('Fitness Values')
plt.title('MIMIC - tuning keeping percentage')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\nqueens-mimic2.png', dpi=500)
plt.show()

In [ ]:
# mimic - best parameter, test time
t0_clock = time.process_time()
mimic_results = mlrose_hiive.mimic(problem, max_attempts=100, max_iters=1000, pop_size=500, keep_pct=0.4, random_state=seed, curve=True)
t1_clock = time.process_time()
print("fitness score: ", mimic_results[1])
print("# iterations: ", mimic_results[2].shape[0])
print("# cumulative function iterations: ", mimic_results[2][-1,1] - mimic_results[2][0,1])
print("time_consumed: ", t1_clock - t0_clock)